In [5]:
#  1. Imports
import yfinance as yf
import pandas as pd
from datetime import datetime
import pytz
import warnings

#  Suppress yfinance FutureWarnings
warnings.simplefilter(action='ignore', category=FutureWarning)

#  2. Define the watchlist
watchlist = ['AMD', 'NVDA', 'TSLA', 'AAPL', 'META', 'MSFT']

#  3. Define market open time (convert 9:30 AM Eastern to UTC)
eastern = pytz.timezone("US/Eastern")
market_open_est = datetime.now(eastern).replace(hour=9, minute=30, second=0, microsecond=0)
market_open_utc = market_open_est.astimezone(pytz.utc)

#  4. Function to pull premarket intraday data
def get_premarket_data(ticker):
    """
    Downloads 1-minute intraday data for the current day including pre/post market.
    Filters rows that occurred before the market officially opened.
    """
    data = yf.download(ticker, period='1d', interval='1m', prepost=True, progress=False)
    premarket_data = data[data.index < market_open_utc]
    return premarket_data

#  5. Loop through each ticker and calculate % change + volume
results = []

for ticker in watchlist:
    df = get_premarket_data(ticker)

    if not df.empty:
        try:
            # Safely extract first open and last close before market open
            open_price = df['Open'].iloc[0].item()
            last_price = df['Close'].iloc[-1].item()

            # Calculate premarket % change
            pct_change = ((last_price - open_price) / open_price) * 100

            # Sum total premarket volume
            volume = df['Volume'].sum().item()

            # Append to results
            results.append({
                'Ticker': ticker,
                'Premarket % Change': round(pct_change, 2),
                'Volume': volume
            })

        except Exception as e:
            print(f"Error processing {ticker}: {e}")

#  6. Build DataFrame and sort by biggest premarket movers
premarket_df = pd.DataFrame(results)

if not premarket_df.empty:
    premarket_df = premarket_df.sort_values(by='Premarket % Change', ascending=False)

#  7. Show the results
premarket_df

Ticker  Premarket % Change     Volume
5   MSFT                1.12   13917734
1   NVDA                1.07  142313659
3   AAPL                0.35   34697317
4   META                0.34    8601653
0    AMD               -1.09   28174157
2   TSLA               -1.73   58042302

In [6]:
#  1. Set your filter thresholds
MIN_MOVE = 2.0       # percent
MIN_VOLUME = 1_000_000  # shares

#  2. Define function to tag setups
def tag_trade_setup(row):
    pct = row['Premarket % Change']
    vol = row['Volume']

    if pct >= MIN_MOVE and vol >= MIN_VOLUME:
        return 'Gap & Go'
    elif pct <= -MIN_MOVE and vol >= MIN_VOLUME:
        return 'Reversal'
    else:
        return 'Low Signal'

#  3. Apply filter and tag
filtered_df = premarket_df[
    (abs(premarket_df['Premarket % Change']) >= MIN_MOVE) &
    (premarket_df['Volume'] >= MIN_VOLUME)
].copy()

filtered_df['Setup'] = filtered_df.apply(tag_trade_setup, axis=1)

#  4. Show only high-probability setups
filtered_df.reset_index(drop=True, inplace=True)
filtered_df

Empty DataFrame
Columns: [Ticker, Premarket % Change, Volume, Setup]
Index: []

In [7]:
from yahoo_fin import news
news.get_yf_rss("TSLA")  # quick test

[{'summary': 'More than a decade ago, Warren Buffett said the national deficit could be solved in just five minutes. His plan? "You just pass a law that says that any time there is a deficit of more than 3% of GDP, all sitting members of Congress are ineligible for reelection." That 2011 quote, from an interview with CNBC\'s Becky Quick, resurfaced in early June — and this time, it gained traction with one of the world\'s most influential voices. On June 4, Utah Sen. Mike Lee (R-UT) posted the clip on X and ask',
  'summary_detail': {'type': 'text/html',
   'language': None,
   'base': 'https://feeds.finance.yahoo.com/rss/2.0/headline?s=TSLA&region=US&lang=en-US',
   'value': 'More than a decade ago, Warren Buffett said the national deficit could be solved in just five minutes. His plan? "You just pass a law that says that any time there is a deficit of more than 3% of GDP, all sitting members of Congress are ineligible for reelection." That 2011 quote, from an interview with CNBC\'s B

In [8]:
# Save the final filtered_df to CSV
filtered_df.to_csv("daily_report.csv", index=False)